In [ ]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [ ]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



Loading transit DataFrame from data/gtfs\transit_df.pkl
Loading stops DataFrame from data/gtfs\stops_df.pkl


In [ ]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk.pkl", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit("data/graphs/ZMG_transit.pkl", stops_df)

Loading walking graph from data/graphs/ZMG_walk.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl


In [ ]:
route_service = RouteService(graph_walk, graph_transit, stops_df, transit_df)

In [ ]:
import networkx as nx

def graph_is_fully_connected(G):
    if len(G.nodes) == 0:
        return True  # empty graph is trivially connected

    start = next(iter(G.nodes))             # pick any node
    visited = set(nx.dfs_preorder_nodes(G, start))
    return len(visited) == len(G.nodes)

def disconnected_nodes(G):
    start = next(iter(G.nodes))
    visited = set(nx.dfs_preorder_nodes(G, start))
    return set(G.nodes) - visited


print(disconnected_nodes(graph_transit))


{'mxc_C111V1_STP_01'}


In [ ]:
import random as rd
from shapely.geometry import Point

#choose randomly a node from the largest connected component
def random_conected_walking_nodes():

    connected = False
    attempts = 0

    while not connected:
        # Pick from largest connected component
        start = rd.choice(list(graph_walk.nodes))
        destination = rd.choice(list(graph_walk.nodes))
        
        # Since they're both in the same connected component, they should be connected
        connected = nx.has_path(graph_walk, source=start, target=destination)
        attempts += 1
        print(f"Attempt {attempts}")
        if attempts > 100:  # Much fewer attempts needed now
            raise Exception("Failed to find connected nodes after 100 attempts")
            
    return start, destination

walk_node_start, walk_node_destination = random_conected_walking_nodes()

#measure distance between the two nodes
distance = ox.distance.euclidean(
    graph_walk.nodes[walk_node_start]['y'],
    graph_walk.nodes[walk_node_start]['x'],
    graph_walk.nodes[walk_node_destination]['y'],
    graph_walk.nodes[walk_node_destination]['x']
)

#closest stop from the walking node start and destination

p = Point(graph_walk.nodes[walk_node_start]['x'], graph_walk.nodes[walk_node_start]['y'])
stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)
nearest_stop_idx = stops_gdf.geometry.distance(p).idxmin()
nearest_stop = stops_gdf.loc[nearest_stop_idx]['stop_id']
distance_to_stop = p.distance(stops_gdf.loc[nearest_stop_idx].geometry)
print("Nearest stop to start walking node:", nearest_stop)
print("Distance to nearest stop:", distance_to_stop)

start = nearest_stop

transit_point = graph_transit.nodes[nearest_stop]['pos']
nearest_walk_node = ox.distance.nearest_nodes(graph_walk, transit_point.x, transit_point.y)
print("Nearest walking node to start transit stop:", nearest_walk_node)

print("************start and destination have a path in walking graph:", nx.has_path(graph_walk, source=walk_node_start, target=nearest_walk_node))

#create a route from walking node start to nearest_walk_node
path = ox.shortest_path(graph_walk, walk_node_start, nearest_walk_node, weight='length')

#create a list of coordinates from the path
path_coords = [(graph_walk.nodes[n]['x'], graph_walk.nodes[n]['y']) for n in path]
#create a LineString from the coordinates
from shapely.geometry import LineString
line = LineString(path_coords)
#create a GeoDataFrame from the LineString
gdf = gpd.GeoDataFrame(geometry=[line], crs="EPSG:3857")

m = gdf.explore()
openMap(m)



p = Point(graph_walk.nodes[walk_node_destination]['x'], graph_walk.nodes[walk_node_destination]['y'])
stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)
nearest_stop_idx = stops_gdf.geometry.distance(p).idxmin()
nearest_stop = stops_gdf.loc[nearest_stop_idx]['stop_id']
distance_to_stop = p.distance(stops_gdf.loc[nearest_stop_idx].geometry)
print("Nearest stop to destination walking node:", nearest_stop)
print("Distance to nearest stop:", distance_to_stop)

destination = nearest_stop

print("start and destination have a path in transit graph:", nx.has_path(graph_transit, source=start, target=destination))




print(f"xStart node: {graph_walk.nodes[walk_node_start]['x']}, yStart node: {graph_walk.nodes[walk_node_start]['y']}")
print(f"xDestination node: {graph_walk.nodes[walk_node_destination]['x']}, yDestination node: {graph_walk.nodes[walk_node_destination]['y']}")

print(f"Euclidean distance between nodes: {distance} meters")
print(graph_walk.graph)

print("Nearest walking node to start point:", walk_node_start)
print("Nearest walking node to destination point:", walk_node_destination)

print("There is a path between the two nodes:", nx.has_path(graph_walk, source=walk_node_start, target=walk_node_destination))

Attempt 1
Nearest stop to start walking node: mxc_C27_STP_136
Distance to nearest stop: 14.526549414790153
Nearest walking node to start transit stop: 1700344684
************start and destination have a path in walking graph: True
Nearest walking node to start transit stop: 1700344684
************start and destination have a path in walking graph: True


GEOSException: IllegalArgumentException: point array must contain 0 or >1 elements


In [ ]:
dijkstra_path, dijkstra_cost = route_service.dijkstra_transit(start, destination)


print("Shortest path from", start, "to", destination, ":", dijkstra_path)

print("Cost of the path:", dijkstra_cost/60)

transit_gdf = gpd.GeoDataFrame(transit_df, geometry='shape_geometry', crs='EPSG:4326')

dijkstra_path_stops = []
dijkstra_path_shapes = []
prev_shape = None
for stop, shape in dijkstra_path:
    dijkstra_path_stops.append(stop)
    dijkstra_path_shapes.append(shape)
    if prev_shape is None:
        print("Walk to stop:", stop)
    if shape == 'walking':
        print("Walk to stop:", stop)
    elif shape != prev_shape:
        print("Take Bus:", shape, "from stop:", stop, " direction to:", transit_df[transit_df['shape_id'] == shape]['trip_headsign'].iloc[0])
    prev_shape = shape


#show all the shapes in the path
#assign colors based on shape_id or randomly
shapes_in_path = sorted(set(dijkstra_path_shapes))
subset = transit_gdf[transit_gdf['shape_id'].isin(shapes_in_path)]
m = subset.explore(
    m=m,
    column='route_long_name',
    cmap='tab20',
    legend=True,
    tooltip=['route_long_name', 'route_short_name', 'trip_headsign'],
    style_kwds={'weight': 6, 'opacity': 0.9}
)
print(set(shapes_in_path))

stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

path_gdf = stops_gdf[stops_gdf['stop_id'].isin(dijkstra_path_stops)]
m = path_gdf.explore(color='orange', m=m)
openMap(m)

Shortest path from mxc_C112_STP_01 to mxc_C94_STP_01 : [('mxc_C112_STP_02', 'C112_r1'), ('mxa_T13AC01_STP_11', 'C112_r1'), ('mxc_C112_STP_04', 'C112_r1'), ('mxc_C112_STP_05', 'C112_r1'), ('mxc_C112_STP_06', 'C112_r1'), ('mxc_C112_STP_07', 'C112_r1'), ('mxc_C112_STP_08', 'C112_r1'), ('mxc_C112_STP_09', 'C112_r1'), ('mxc_C112_STP_10', 'C112_r1'), ('mxc_C67V1_STP_44', 'C112_r1'), ('mxc_C102_STP_76', 'C47-V2_r1'), ('mxc_C102_STP_77', 'C47-V2_r1'), ('mxc_C111V1_STP_32', 'C47-V2_r1'), ('mxc_C111V1_STP_33', 'C47-V2_r1'), ('mxc_C111V1_STP_34', 'C47-V2_r1'), ('mxc_C111V1_STP_35', 'C47-V2_r1'), ('mxc_C111V1_STP_36', 'C47-V2_r1'), ('mxc_C67V1_STP_51', 'C47-V2_r1'), ('mxc_C67V1_STP_52', 'C47-V2_r1'), ('mxc_C67V1_STP_53', 'C47-V2_r1'), ('mxc_C97V2_STP_28', 'walking'), ('mxc_C97V2_STP_29', 'T18-2-T_r2'), ('mxd_mxc_AMG_T18_2_STP_13', 'T18-2-T_r2'), ('mxc_C60_STP_58', 'T18-2-T_r2'), ('mxd_mxc_AMG_T18_2_STP_15', 'T18-2-T_r2'), ('mxd_mxc_AMG_T18_2_STP_16', 'T18-2-T_r2'), ('mxd_mxc_AMG_T18_2_STP_17', 'T1

In [ ]:

import folium


start = Point(graph_walk.nodes[walk_node_start]['x'], graph_walk.nodes[walk_node_start]['y'])
destination = Point(graph_walk.nodes[walk_node_destination]['x'], graph_walk.nodes[walk_node_destination]['y'])

total_time, m_comb = route_service.route_combined(start, destination)
# Add layer control
folium.LayerControl().add_to(m_comb)

openMap(m_comb)